In [1]:
import numpy as np
# constant size
size = 1000
# Set p(H) apriori for simulation as a biased coin.
p_h = 0.1

# Model experiment as a single biased coin flipped 1000 times.
data = np.random.binomial(1, p_h, size=size)

In [2]:
n_states = 2
n_obs = 2

# emission = np.random.rand(size,n_states)
# emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))

emission = np.random.rand(n_states,n_obs)
emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))
# emission = np.array([[0.5,0.5],[0.5,0.5]])

transition =  np.random.rand(n_states,n_states)
transition = transition/np.tile(np.expand_dims(np.sum(transition,axis=1),axis=1),(1,2))
# transition = np.array([[0.3,0.7],[0.4,0.6]])


forward = np.random.rand(size,n_states)
scale_factors = np.zeros((size))
forward_hat = np.zeros((size,n_states))

backward = np.random.rand(size,n_states)
backward_hat = np.zeros((size,n_states))

init_prob = np.array([0.5,0.5])


p_old = -10000
tol = 0.0001
max_iter = 100

mu = np.random.rand(n_states)

In [3]:
for ite in range(max_iter):
    forward[0,:] = init_prob * emission[:,data[0]]
    scale_factors[0] = np.sum(forward[0,:])
    forward_hat[0,:] = forward[0,:]/scale_factors[0]
    
    for t in range(size-1):
        temp = np.matmul(forward_hat[t,:] ,transition) * emission[:,data[t+1]]
        scale_factors[t+1] = np.sum(temp)
        forward_hat[t+1,:] = temp/scale_factors[t+1]
        forward[t+1,:] = forward_hat[t+1,:]*np.prod(scale_factors[0:t+1])

    backward[-1,:] = 1
    backward_hat[-1,:] = backward[-1,:]
    for t in reversed(range(size-1)):
        temp = np.matmul(backward_hat[t+1,:]*emission[:,data[t+1]],np.transpose(transition))
        backward_hat[t,:] = temp/scale_factors[t+1]
        backward[t,:] = backward_hat[t,:]*np.prod(scale_factors[t+1:-1])


    a = np.zeros((size,n_states))
    b = np.zeros((size,n_states,n_states))
    for i in range(size):
        for j in range(n_states):
            a[i,j] = forward_hat[i,j]*backward_hat[i,j]
    for t in range(size-1):
        for i in range(n_states):
            for j in range(n_states):
                b[t,i,j] = scale_factors[t+1]*forward_hat[t,i]*backward_hat[t+1,j]*transition[i,j]*emission[j,data[t+1]]

    for i in range(n_states):
        for j in range(n_states):
            transition[i,j] = np.sum(b[0:-1,i,j])/np.sum(b[0:-1,i,:])

    for i in range(n_states):
        init_prob[i] = a[0,i]/np.sum(a[0,:])

    for i in range(n_states):
        for j in range(n_states):
            emission[j,i] = np.sum(a[np.argwhere(data==i),j]) / np.sum(a[:,j])
            
    p = np.sum(np.log(scale_factors))
    print(f'p is:{p}')
    # print(f'transition prob is: {transition}')
    # print(f'emission prob is:{emission}')
    # if p - p_old < tol:
    #     break
    p_old = p

p is:-1115.4742902275439
p is:-309.42587853975215
p is:-309.42580933933414
p is:-309.4257409461097
p is:-309.42567286378596
p is:-309.42560508584916
p is:-309.4255376054301
p is:-309.42547041533203
p is:-309.4254035080485
p is:-309.4253368757872
p is:-309.4252705104917
p is:-309.42520440386056
p is:-309.42513854736893
p is:-309.4250729322896
p is:-309.42500754971013
p is:-309.424942390555
p is:-309.42487744560253
p is:-309.4248127055048
p is:-309.4247481608062
p is:-309.4246838019589
p is:-309.4246196193436
p is:-309.42455560328534
p is:-309.4244917440701
p is:-309.42442803196235
p is:-309.42436445722086
p is:-309.42430101011666
p is:-309.42423768094625
p is:-309.42417446004885
p is:-309.42411133782275
p is:-309.4240483047391
p is:-309.4239853513574
p is:-309.42392246834186
p is:-309.423859646475
p is:-309.42379687667204
p is:-309.4237341499969
p is:-309.42367145767537
p is:-309.42360879111084
p is:-309.4235461418971
p is:-309.4234835018341
p is:-309.4234208629408
p is:-309.42335821747

In [4]:
emission

array([[0.90359514, 0.09640486],
       [0.91678192, 0.08321808]])

In [5]:
transition

array([[0.72922994, 0.27077006],
       [0.76874047, 0.23125953]])

In [6]:
from hmmlearn import hmm

In [22]:
model = hmm.CategoricalHMM(n_components=n_states,emissionprob_prior=emission,transmat_prior=transition,startprob_prior=init_prob)


In [23]:
model.fit(data.reshape(-1,1))
hidden_states = model.predict(data.reshape(-1,1))
print("Most likely hidden states:", hidden_states)


Most likely hidden states: [0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [24]:
data

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,

In [25]:
model.emissionprob_

array([[1.        , 0.        ],
       [0.83837384, 0.16162616]])

In [26]:
model.transmat_

array([[0.59556921, 0.40443079],
       [0.30389523, 0.69610477]])